# Install necessary libraries

In [ ]:
pip install openai==0.28

In [ ]:
!pip install openai faiss-cpu tiktoken

In [ ]:
import openai
import faiss
import numpy as np
import tiktoken

# Set your OpenAI API key

In [ ]:
openai.api_key = ''

# Sample documents (replace with real knowledge base)

In [ ]:
documents = [
    "Machine learning is a subset of artificial intelligence.",
    "The Eiffel Tower is located in Paris, France.",
    "Python is a popular programming language for data science."
]

# Tokenizer for text embedding

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")

# Generate embeddings for the documents

In [ ]:
def get_embedding(text):
    response = openai.Embedding.create(model="text-embedding-ada-002", input=text)
    return response["data"][0]["embedding"]

document_embeddings = np.array([get_embedding(doc) for doc in documents])

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

# Create FAISS index for efficient retrieval

In [ ]:
index = faiss.IndexFlatL2(len(document_embeddings[0]))
index.add(document_embeddings)

NameError: name 'document_embeddings' is not defined

# Function to retrieve the most relevant document

In [ ]:
def retrieve_relevant_document(query):
    query_embedding = np.array([get_embedding(query)], dtype=np.float32)
    _, indices = index.search(query_embedding, 1)  # Retrieve top 1 document
    return documents[indices[0][0]]

# Chatbot function using RAG

In [ ]:
def chat_with_rag(query):
    relevant_doc = retrieve_relevant_document(query)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant that uses retrieved documents to enhance responses."},
            {"role": "user", "content": f"Context: {relevant_doc}\nUser Question: {query}"}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Interactive chat loop

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    response = chat_with_rag(user_input)
    print(f"Bot: {response}")